In [364]:
import tkinter as tk
from tkinter import messagebox
import csv
import logging

In [365]:
# Configure logging for clear, minimal warnings
logging.basicConfig(level=logging.WARNING, format='%(levelname)s:%(message)s')

def load_data(csv_path="data.csv"):
    """Load user, feedback, and report data from a CSV file."""
    users = []
    feedback_data = []
    report_data = []

    try:
        with open(csv_path, mode="r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                row_type = row.get("type")
                
                # Process user data
                if row_type == "user":
                    users.append({
                        "name": row["username"],
                        "password": row["password"],
                        "role": row["role"]
                    })
                
                # Process feedback data
                elif row_type == "feedback":
                    feedback_data.append({
                        "feedback_id": row["feedback_id"],
                        "agent": row["agent"],
                        "feedback_text": row["feedback_text"],
                        "status": row["status"]
                    })
                
                # Process report data
                elif row_type == "report":
                    report_data.append({
                        "report_id": row["report_id"],
                        "date": row["date"],
                        "agent": row["agent"],
                        "feedback_applied": row["feedback_applied"]
                    })

    except FileNotFoundError:
        print(f"Error: CSV file '{csv_path}' not found.")
    except IOError:
        print(f"Error reading the file '{csv_path}'.")

    # Temporary debugging prints for confirmation
    print("Loaded users:", users)
    print("Loaded feedback data:", feedback_data)
    print("Loaded report data:", report_data)
    
    return users, feedback_data, report_data

In [366]:
def save_data(users, feedback_data, report_data, csv_path="data.csv"):
    # Define the order of the columns for consistency
    fieldnames = [
        "type", "username", "password", "role", 
        "feedback_id", "agent", "feedback_text", "status", 
        "report_id", "date", "feedback_applied", 
        "feedback_entry_date", "integration_date"
    ]
    
    # Prepare data for writing to CSV
    data_to_write = []
    
    # Convert users list to list of dictionaries with "user" type
    for user in users:
        data_to_write.append({
            "type": "user",
            "username": user["name"],
            "password": user["password"],
            "role": user["role"],
            "feedback_id": "",
            "agent": "",
            "feedback_text": "",
            "status": "",
            "report_id": "",
            "date": "",
            "feedback_applied": "",
            "feedback_entry_date": "",
            "integration_date": ""
        })
    
    # Add feedback and report data as before
    for feedback in feedback_data:
        data_to_write.append({
            "type": "feedback",
            "username": "",
            "password": "",
            "role": "",
            "feedback_id": feedback["feedback_id"],
            "agent": feedback["agent"],
            "feedback_text": feedback["feedback_text"],
            "status": feedback["status"],
            "report_id": "",
            "date": "",
            "feedback_applied": "",
            "feedback_entry_date": feedback.get("feedback_entry_date", ""),
            "integration_date": feedback.get("integration_date", "")
        })
    
    for report in report_data:
        data_to_write.append({
            "type": "report",
            "username": "",
            "password": "",
            "role": "",
            "feedback_id": "",
            "agent": report["agent"],
            "feedback_text": "",
            "status": "",
            "report_id": report["report_id"],
            "date": report["date"],
            "feedback_applied": report["feedback_applied"],
            "feedback_entry_date": "",
            "integration_date": ""
        })
    
    # Write all data to CSV
    with open(csv_path, mode="w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data_to_write)


In [367]:
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Agent and Coach App")
        self.geometry("400x300")

        # Load data once and store it in attributes
        self.users, self.feedback_data, self.report_data = load_data()
        
        # Debugging print to verify loaded feedback data
        print("Loaded feedback data:", self.feedback_data)  

        # Container frame to hold all screens
        container = tk.Frame(self)
        container.pack(fill="both", expand=True)
        
        # Dictionary to store frames
        self.frames = {}
        
        # Add each screen frame to the dictionary
        for F in (LoginScreen, AgentMainMenu, CoachMainMenu, ProfileSettings, ViewFeedback, AgentViewFeedback):
            frame = F(container, self)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")

        # Start with the login screen
        self.show_frame(LoginScreen)

    def show_frame(self, frame_class):
        """Show a frame for the given class."""
        frame = self.frames[frame_class]
        frame.tkraise()

In [368]:
class LoginScreen(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Login Screen").pack(pady=10)
        
        # Role selection
        self.role = tk.StringVar(value="Agent")
        tk.Radiobutton(self, text="Agent", variable=self.role, value="Agent").pack()
        tk.Radiobutton(self, text="Coach", variable=self.role, value="Coach").pack()
        
        # Username and Password
        tk.Label(self, text="Username").pack()
        self.username_entry = tk.Entry(self)
        self.username_entry.pack()

        tk.Label(self, text="Password").pack()
        self.password_entry = tk.Entry(self, show="*")
        self.password_entry.pack()
        
        # Login button
        login_button = tk.Button(self, text="Login", command=self.login)
        login_button.pack()

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        role = self.role.get()

        # Search for the user in the list
        user_found = None
        for user in self.controller.users:
            if user["name"] == username and user["password"] == password and user["role"] == role:
                user_found = user
                break

        # Validate the found user
        if user_found:
            self.controller.current_user = username  # Set current_user upon successful login
            if role == "Agent":
                self.controller.show_frame(AgentMainMenu)
            elif role == "Coach":
                self.controller.show_frame(CoachMainMenu)
        else:
            messagebox.showerror("Login Error", "Username not found or incorrect credentials.")

In [369]:
class AgentMainMenu(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Agent Main Menu").pack(pady=10)
        
        # Profile and View Feedback buttons
        profile_button = tk.Button(self, text="Profile", command=lambda: controller.show_frame(ProfileSettings))
        profile_button.pack()
        
        # Feedback buttons
        view_feedback_button = tk.Button(self, text="View Feedback", command=lambda: controller.show_frame(AgentViewFeedback))
        view_feedback_button.pack()

        logout_button = tk.Button(self, text="Logout", command=lambda: controller.show_frame(LoginScreen))
        logout_button.pack()

In [370]:
class CoachMainMenu(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Coach Main Menu").pack(pady=10)
        
        # View Profile button
        profile_button = tk.Button(self, text="View Profile", command=lambda: controller.show_frame(ProfileSettings))
        profile_button.pack(pady=5)
        
        # Create Account
        create_account_button = tk.Button(self, text="Create Account", command=self.open_create_account_form)
        create_account_button.pack()
        
        # Update Feedback button to navigate to ViewFeedback
        update_feedback_button = tk.Button(self, text="Update Feedback", command=lambda: controller.show_frame(ViewFeedback))
        update_feedback_button.pack(pady=5)
        
        # Input Feedback
        input_feedback_button = tk.Button(self, text="Input Feedback", command=self.input_feedback)
        input_feedback_button.pack()

        # Generate Reports buttons
        generate_reports_button = tk.Button(self, text="Generate Reports", command=self.generate_reports)
        generate_reports_button.pack()

        logout_button = tk.Button(self, text="Logout", command=lambda: controller.show_frame(LoginScreen))
        logout_button.pack()

    def open_create_account_form(self):
        # Open the account creation form in a new window
        create_account_window = tk.Toplevel(self)
        create_account_window.title("Create New Account")
        
        # Username input
        tk.Label(create_account_window, text="Username").grid(row=0, column=0, padx=10, pady=5)
        username_entry = tk.Entry(create_account_window)
        username_entry.grid(row=0, column=1, padx=10, pady=5)

        # Password input
        tk.Label(create_account_window, text="Password").grid(row=1, column=0, padx=10, pady=5)
        password_entry = tk.Entry(create_account_window, show="*")
        password_entry.grid(row=1, column=1, padx=10, pady=5)

        # Role selection (default to "Agent")
        tk.Label(create_account_window, text="Role").grid(row=2, column=0, padx=10, pady=5)
        role_var = tk.StringVar(value="Agent")
        role_menu = tk.OptionMenu(create_account_window, role_var, "Agent", "Coach")
        role_menu.grid(row=2, column=1, padx=10, pady=5)

        # Submit button
        submit_button = tk.Button(
            create_account_window, 
            text="Create Account", 
            command=lambda: self.create_account(username_entry.get(), password_entry.get(), role_var.get(), create_account_window)
        )
        submit_button.grid(row=3, columnspan=2, pady=10)

    def create_account(self, username, password, role, window):
        # Check for empty fields
        if not username or not password:
            messagebox.showerror("Input Error", "Username and password cannot be empty.")
            return

        # Check if username already exists
        if any(user['name'] == username for user in self.controller.users):
            messagebox.showerror("Input Error", "Username already exists.")
            return

        # Add new user to the users list
        self.controller.users.append({"name": username, "password": password, "role": role})

        # Save changes to the CSV file
        save_data(self.controller.users, self.controller.feedback_data, self.controller.report_data)

        # Close the account creation form
        window.destroy()
        messagebox.showinfo("Account Created", f"Account for {username} created successfully.")

    def input_feedback(self):
        # Placeholder for Input Feedback functionality
        messagebox.showinfo("Input Feedback", "Input Feedback screen (not implemented).")

    def generate_reports(self):
        # Placeholder for Generate Reports functionality
        messagebox.showinfo("Generate Reports", "Generate Reports screen (not implemented).")

In [371]:
class ProfileSettings(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Profile Settings").pack(pady=10)

        # Password fields
        tk.Label(self, text="Current Password").pack()
        self.current_password = tk.Entry(self, show="*")
        self.current_password.pack()

        tk.Label(self, text="New Password").pack()
        self.new_password = tk.Entry(self, show="*")
        self.new_password.pack()

        # Save button
        save_button = tk.Button(self, text="Save Changes", command=self.update_password)
        save_button.pack(pady=5)

        # Back button to return to the main menu
        back_button = tk.Button(self, text="Back to Main Menu", command=self.go_back)
        back_button.pack(pady=5)

    def update_password(self):
        username = self.controller.current_user  # Use the logged-in user’s username
        current_pw = self.current_password.get()
        new_pw = self.new_password.get()

        # Locate the user in the users list and update the password if the current password matches
        user_found = False
        for user in self.controller.users:
            if user["name"] == username and user["password"] == current_pw:
                user["password"] = new_pw
                user_found = True
                break

        if user_found:
            try:
                save_data(self.controller.users, self.controller.feedback_data, self.controller.report_data)
                messagebox.showinfo("Profile", "Password updated successfully.")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save changes: {e}")
        else:
            messagebox.showerror("Error", "Current password is incorrect.")

    def go_back(self):
        """Navigate back to the correct main menu based on the current user's role."""
        # Find the current user's role
        current_user = self.controller.current_user
        user_role = next((user["role"] for user in self.controller.users if user["name"] == current_user), None)

        # Navigate to the correct frame based on the role
        if user_role == "Coach":
            self.controller.show_frame(CoachMainMenu)
        elif user_role == "Agent":
            self.controller.show_frame(AgentMainMenu)
        else:
            messagebox.showerror("Navigation Error", "Unable to determine user role for navigation.")

In [372]:
class ViewFeedback(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Coach: View and Update Feedback").pack(pady=10)

        # Dropdown menu to select an agent
        self.agent_var = tk.StringVar(value="Select Agent")
        agent_names = [user['name'] for user in self.controller.users if user.get('role') == 'Agent']
        
        # Frame to hold the dropdown
        agent_frame = tk.Frame(self)
        agent_frame.pack(pady=5)
        
        self.agent_dropdown = tk.OptionMenu(agent_frame, self.agent_var, *agent_names, command=self.filter_feedback_by_agent)
        self.agent_dropdown.grid(row=0, column=0, padx=5)

        # Listbox to display feedback
        self.feedback_listbox = tk.Listbox(self, width=50, height=10)
        self.feedback_listbox.pack(pady=5)
        self.feedback_listbox.bind("<<ListboxSelect>>", self.on_feedback_select)

        # Status dropdown to assign feedback status
        tk.Label(self, text="Assign Status").pack(pady=5)
        self.status_var = tk.StringVar(value="Not Integrated")
        self.status_dropdown = tk.OptionMenu(self, self.status_var, "Not Integrated", "Integrated")
        self.status_dropdown.pack()

        # Save button to update feedback status
        save_button = tk.Button(self, text="Save Status", command=self.update_feedback_status)
        save_button.pack(pady=10)

        # Back button to return to the main menu
        back_button = tk.Button(self, text="Back to Main Menu", command=lambda: controller.show_frame(CoachMainMenu))
        back_button.pack(pady=5)

        # Ensure feedback list starts empty by calling populate with an empty list
        self.populate_feedback_list([])

    def populate_feedback_list(self, feedback_data=None):
        """Populate the listbox with feedback entries, optionally filtered by agent."""
        self.feedback_listbox.delete(0, tk.END)
        
        # Use the provided feedback_data or default to an empty list if none provided
        feedbacks = feedback_data if feedback_data is not None else []

        if feedbacks:
            for feedback in feedbacks:
                feedback_text = f"{feedback['feedback_id']}: {feedback['feedback_text']} (Status: {feedback['status']})"
                self.feedback_listbox.insert(tk.END, feedback_text)
        else:
            self.feedback_listbox.insert(tk.END, "No feedback available for this agent.")

    def filter_feedback_by_agent(self, selected_agent):
        """Filter feedback entries to display only those related to the selected agent."""
        # Clear the list if no agent is selected
        if selected_agent == "Select Agent" or not selected_agent:
            self.populate_feedback_list([])  # Clear the feedback list
            return

        # Filter feedback based on the selected agent
        filtered_feedback = [fb for fb in self.controller.feedback_data if fb.get('agent') == selected_agent]
        self.populate_feedback_list(filtered_feedback)

    def on_feedback_select(self, event):
        """Handle selection of feedback in the listbox and set the status dropdown to the current status."""
        try:
            selected_index = self.feedback_listbox.curselection()[0]
            selected_feedback = self.controller.feedback_data[selected_index]
            self.status_var.set(selected_feedback["status"])  # Set dropdown to current feedback status
        except IndexError:
            pass  # No item selected

    def update_feedback_status(self):
        """Update the selected feedback's status and save it to the CSV."""
        try:
            selected_index = self.feedback_listbox.curselection()[0]
            selected_feedback = self.controller.feedback_data[selected_index]
            selected_feedback["status"] = self.status_var.get()  # Update status from dropdown

            # Call save_data to write updates back to the CSV
            save_data(self.controller.users, self.controller.feedback_data, self.controller.report_data)
            messagebox.showinfo("Success", "Feedback status updated successfully.")
            self.populate_feedback_list()  # Refresh the list display
        except IndexError:
            messagebox.showerror("Selection Error", "Please select a feedback entry to update.")

In [373]:
class AgentViewFeedback(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Agent: View Feedback").pack(pady=10)
        
        # Dropdown to filter feedback by status
        tk.Label(self, text="Filter by Status").pack(pady=5)
        self.filter_var = tk.StringVar(value="All")
        self.filter_dropdown = tk.OptionMenu(self, self.filter_var, "All", "Integrated", "Not Integrated", command=self.filter_feedback)
        self.filter_dropdown.pack(pady=5)

        # Listbox to display feedback based on filter
        self.feedback_listbox = tk.Listbox(self, width=50, height=10)
        self.feedback_listbox.pack(pady=10)

        # Refresh button to reload latest CSV data
        refresh_button = tk.Button(self, text="Refresh Feedback", command=self.refresh_feedback)
        refresh_button.pack(pady=5)

        # Back button to return to the main menu (agent view only)
        back_button = tk.Button(self, text="Back to Main Menu", command=lambda: controller.show_frame(AgentMainMenu))
        back_button.pack(pady=5)

        # Populate the feedback list initially with all entries
        self.populate_feedback_list("All")

    def populate_feedback_list(self, filter_status="All"):
        """Display feedback entries filtered by status in the listbox."""
        self.feedback_listbox.delete(0, tk.END)  # Clear the listbox

        # Filter feedback data based on selected status
        feedback_data = self.controller.feedback_data
        if filter_status != "All":
            feedback_data = [fb for fb in feedback_data if fb["status"] == filter_status]

        # Display filtered feedback entries in the listbox
        for feedback in feedback_data:
            feedback_text = f"{feedback['feedback_id']}: {feedback['feedback_text']} (Status: {feedback['status']})"
            self.feedback_listbox.insert(tk.END, feedback_text)

    def filter_feedback(self, *args):
        """Filter feedback based on the selected status in the dropdown menu."""
        selected_filter = self.filter_var.get()
        self.populate_feedback_list(selected_filter)

    def refresh_feedback(self):
        """Reload feedback data from CSV and repopulate list based on current filter."""
        # Reload the data from CSV using load_data()
        self.controller.users, self.controller.feedback_data, self.controller.report_data = load_data()
        # Apply the current filter after refreshing
        self.populate_feedback_list(self.filter_var.get())

In [374]:
if __name__ == "__main__":
    app = App()
    app.mainloop()

Loaded users: [{'name': 'agent1', 'password': 'pass987', 'role': 'Agent'}, {'name': 'coach1', 'password': 'pass456', 'role': 'Coach'}, {'name': 'agent4', 'password': 'pass123', 'role': 'Agent'}, {'name': 'coach2', 'password': 'pass654', 'role': 'Coach'}]
Loaded feedback data: [{'feedback_id': 'feedback_01', 'agent': 'agent1', 'feedback_text': 'Great job on greeting customers!', 'status': 'Not Integrated'}, {'feedback_id': 'feedback_02', 'agent': 'agent1', 'feedback_text': 'Needs improvement in closing conversations.', 'status': 'Not Integrated'}]
Loaded report data: [{'report_id': 'report_01', 'date': '2024-11-12', 'agent': 'agent1', 'feedback_applied': 'Summary of feedback applied'}]
Loaded feedback data: [{'feedback_id': 'feedback_01', 'agent': 'agent1', 'feedback_text': 'Great job on greeting customers!', 'status': 'Not Integrated'}, {'feedback_id': 'feedback_02', 'agent': 'agent1', 'feedback_text': 'Needs improvement in closing conversations.', 'status': 'Not Integrated'}]
Loaded u